# 问题形式化

In [1]:
class Search:
    # 初始状态(N, N, 1)
    def __init__(self, N, k):
        self.N = N
        self.k = k
        self.visited = []  # 存储访问过的状态
    
    # 目标状态(0, 0, 0)
    def is_goal(self, state):
        if state[0] == state[1] == state[2] == 0:
            return True
        else:
            return False
        
    def is_visited(self, state):
        return True if state in self.visited else False
    
    def push_visited(self, state):
        self.visited.append(state)
        
    def pop_visited(self):
        return self.visited.pop()
        
    def act(self, state, action):
        sign = -1 if state[2] == 1 else 1  # 船在左边，左岸人数减少；船在右边，左岸人数增加
        new_state = state.copy()
        new_state[0] += sign * action[0]
        new_state[1] += sign * action[1]
        new_state[2] = int(not new_state[2])
        # 判断是否是合理动作
        if (0 <= new_state[0] <= self.N and 0 <= new_state[1] <= self.N and 
           (new_state[0] == new_state[1] or new_state[0] == 0 or new_state[0] == self.N)):
            return new_state
        else:
            return None
    
    

In [2]:
N = 5
k = 3
s0 = [N, N, 1]  # (missionary, cannibal, boat)

actions = []  # 渡河方案（动作集合）
for i in range(1, k + 1):   # 载人数 (船上至少要有一人)
    for m in range(i + 1):  # 传教士人数
        c = i - m           # 野人数
        if m == 0 or m >= c:  # 全载野人或传教士多于野人
            actions.append([m, c])

print(actions)

[[0, 1], [1, 0], [0, 2], [1, 1], [2, 0], [0, 3], [2, 1], [3, 0]]


In [3]:
def print_ans(ans):  # [state, action]
    for each in ans:
        if each[0][2] == 0:
            print('{}个传教士和{}个野人从左岸乘船至右岸'.format(each[1][0], each[1][1]))
        else:
            print('{}个传教士和{}个野人从右岸乘船至左岸'.format(each[1][0], each[1][1]))
        print('左岸有{}个传教士和{}个野人'.format(each[0][0], each[0][1]))
        print('右岸有{}个传教士和{}个野人'.format(N - each[0][0], N - each[0][1]))
        print()
        

def print_ans_last(state, last):  # 返回代价，递归过程自增
    node = last[state[0]][state[1]][state[2]]
    last_state = node[0]
    cost = 1
    if last_state != s0:
        cost = print_ans_last(last_state, last) + 1  # 递归实现反序
    if state[2] == 0:
        print('{}个传教士和{}个野人从左岸乘船至右岸'.format(node[1][0], node[1][1]))
    else:
        print('{}个传教士和{}个野人从右岸乘船至左岸'.format(node[1][0], node[1][1]))
    print('左岸有{}个传教士和{}个野人'.format(state[0], state[1]))
    print('右岸有{}个传教士和{}个野人'.format(N - state[0], N - state[1]))
    print()
    return cost

# 无信息搜索

## 宽度优先搜索

In [4]:
from queue import Queue

search = Search(N, k)
search.push_visited(s0)
last = [[[None, None, None] for i in range(N + 1)] for j in range(N + 1)]  # 记录前驱结点 [state, action]
que = Queue()
que.put(s0)
search_cost = 0


def bfs():
    while not que.empty():
        state = que.get()
        global search_cost
        search_cost += 1
        if search.is_goal(state):
            print('answer:')
            cost = print_ans_last(state, last)
            print('cost: ' + str(cost))
            print('search_cost: ' + str(search_cost))
            break
        for action in actions:
            new_state = search.act(state, action)
            if new_state is None or search.is_visited(new_state):
                continue
            last[new_state[0]][new_state[1]][new_state[2]] = [state, action]
            search.push_visited(new_state)
            que.put(new_state)
        

bfs()


answer:
0个传教士和2个野人从左岸乘船至右岸
左岸有5个传教士和3个野人
右岸有0个传教士和2个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有5个传教士和4个野人
右岸有0个传教士和1个野人

0个传教士和3个野人从左岸乘船至右岸
左岸有5个传教士和1个野人
右岸有0个传教士和4个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有5个传教士和2个野人
右岸有0个传教士和3个野人

3个传教士和0个野人从左岸乘船至右岸
左岸有2个传教士和2个野人
右岸有3个传教士和3个野人

1个传教士和1个野人从右岸乘船至左岸
左岸有3个传教士和3个野人
右岸有2个传教士和2个野人

3个传教士和0个野人从左岸乘船至右岸
左岸有0个传教士和3个野人
右岸有5个传教士和2个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和4个野人
右岸有5个传教士和1个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有0个传教士和2个野人
右岸有5个传教士和3个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和3个野人
右岸有5个传教士和2个野人

0个传教士和3个野人从左岸乘船至右岸
左岸有0个传教士和0个野人
右岸有5个传教士和5个野人

cost: 11
search_cost: 26


## 深度优先搜索

In [5]:
search = Search(N, k)
search.push_visited(s0)
ans = []  # [state, action]
search_cost = 0


def dfs(state, cur):
    global search_cost
    search_cost += 1
    if search.is_goal(state):
        print('answer:')
        print_ans(ans)
        print('cost: ' + str(cur))  # 搜索深度就是代价
        print('search_cost: ' + str(search_cost))
        return True
    for action in actions:
        new_state = search.act(state, action)
        if new_state is None or search.is_visited(new_state):
            continue
        search.push_visited(new_state)
        ans.append([new_state, action])
        if dfs(new_state, cur + 1):  # 找到解就直接退出
            return True
        # search.pop_visited()  # 回溯可穷举所有解
        ans.pop()
    return False
        

dfs(s0, 0)


answer:
0个传教士和2个野人从左岸乘船至右岸
左岸有5个传教士和3个野人
右岸有0个传教士和2个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有5个传教士和4个野人
右岸有0个传教士和1个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有5个传教士和2个野人
右岸有0个传教士和3个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有5个传教士和3个野人
右岸有0个传教士和2个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有5个传教士和1个野人
右岸有0个传教士和4个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有5个传教士和2个野人
右岸有0个传教士和3个野人

3个传教士和0个野人从左岸乘船至右岸
左岸有2个传教士和2个野人
右岸有3个传教士和3个野人

1个传教士和1个野人从右岸乘船至左岸
左岸有3个传教士和3个野人
右岸有2个传教士和2个野人

3个传教士和0个野人从左岸乘船至右岸
左岸有0个传教士和3个野人
右岸有5个传教士和2个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和4个野人
右岸有5个传教士和1个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有0个传教士和2个野人
右岸有5个传教士和3个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和3个野人
右岸有5个传教士和2个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有0个传教士和1个野人
右岸有5个传教士和4个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和2个野人
右岸有5个传教士和3个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有0个传教士和0个野人
右岸有5个传教士和5个野人

cost: 15
search_cost: 20


True

# 有信息搜索

## A*算法
$f(n)=g(n)+h(n)$

### 启发式函数(1)：
$当前左岸剩余人数 \\
h(n)=M+C$

In [6]:
def h1(state):
    return state[0] + state[1]

### 启发式函数(2)：
$状态转换后左岸剩余人数 \\
h(n)=M+C-k \cdot B$

In [7]:
# 该启发式函数可能出现负数，但只可能在最后一步出现，因此不会导致后访问的点评价函数更低
def h2(state):
    return state[0] + state[1] - k * state[2]

### 算法主体

In [8]:
from queue import PriorityQueue

h = h2  # which heuristic function

search = Search(N, k)
search.push_visited(s0)
last = [[[None, None, None] for i in range(N + 1)] for j in range(N + 1)]  # 记录前驱结点 [state, action]
que = PriorityQueue()  # [priority, state, last_state, action, g(n)]
que.put([h(s0), s0, None, None, 0])
search_cost = 0  # 搜索代价


def AStar():
    while not que.empty():
        priority, state, last_state, last_action, g_n = que.get()
        # print(priority)
        last[state[0]][state[1]][state[2]] = [last_state, last_action]
        search.push_visited(state)  # 注意放入close list的时机与bfs有所不同
        
        global search_cost
        search_cost += 1
        if search.is_goal(state):
            print('answer:')
            cost = print_ans_last(state, last)
            print('cost: ' + str(cost))
            print('search_cost: ' + str(search_cost))
            break
        for action in actions:
            new_state = search.act(state, action)
            if new_state is None or search.is_visited(new_state):
                continue
            f_n = g_n + 1 + h(new_state)  # evaluation function
            que.put([f_n, new_state, state, action, g_n + 1])
        

AStar()


answer:
0个传教士和3个野人从左岸乘船至右岸
左岸有5个传教士和2个野人
右岸有0个传教士和3个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有5个传教士和3个野人
右岸有0个传教士和2个野人

0个传教士和3个野人从左岸乘船至右岸
左岸有5个传教士和0个野人
右岸有0个传教士和5个野人

0个传教士和2个野人从右岸乘船至左岸
左岸有5个传教士和2个野人
右岸有0个传教士和3个野人

3个传教士和0个野人从左岸乘船至右岸
左岸有2个传教士和2个野人
右岸有3个传教士和3个野人

1个传教士和1个野人从右岸乘船至左岸
左岸有3个传教士和3个野人
右岸有2个传教士和2个野人

3个传教士和0个野人从左岸乘船至右岸
左岸有0个传教士和3个野人
右岸有5个传教士和2个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和4个野人
右岸有5个传教士和1个野人

0个传教士和3个野人从左岸乘船至右岸
左岸有0个传教士和1个野人
右岸有5个传教士和4个野人

0个传教士和1个野人从右岸乘船至左岸
左岸有0个传教士和2个野人
右岸有5个传教士和3个野人

0个传教士和2个野人从左岸乘船至右岸
左岸有0个传教士和0个野人
右岸有5个传教士和5个野人

cost: 11
search_cost: 26
